# Analyse ROMY Events

In [1]:
import os
import numpy as np
import obspy as obs
import matplotlib
import matplotlib.pyplot as plt

from sixdegrees.sixdegrees import sixdegrees

## Configurations

In [2]:
config = {
    # Required time window parameters
    'tbeg': "2025-07-29 23:30:00",  # Start time
    'tend': "2025-08-01 23:30:00",  # End time

    # event
    'origin_time': "2025-07-29 23:24:50",
    'magnitude': 8.8,

    # Station coordinates (required)
    'station_lon': 11.275476,  # BSPF longitude
    'station_lat': 48.162941,    # BSPF latitude
    
    # SEED identifiers
    'seed': "XX.ROMY..",     # Default seed ID
    'rot_seed': ["BW.ROMY.10.BJZ", "BW.ROMY..BJU", "BW.ROMY..BJV"], # "BW.ROMY.30.BJ*",  # Rotation sensor seed ID
    'tra_seed': ["GR.FUR..BHZ", "GR.FUR..BHN", "GR.FUR..BHE"],  # Translation sensor seed ID
    
    # Data source
    'data_source': "sds",  # "fdsn" | "sds" | "mseed_file"

    # Path to local mseed file
    # 'path_to_mseed_file': "./data/romy_eventM6.8.mseed",  # "./data/eventM62_6c.mseed",

    # Path to SDS for rotation and translation data
    'path_to_sds_rot': "/home/andbro/freenas/romy_archive/",    # SDS path for rotation data
    'path_to_sds_tra': "/home/andbro/bay200/mseed_online/archive/",  # SDS path for translation data

    # Path to inventory of rotation and translation sensor
    'path_to_inv_rot': "/home/andbro/kilauea-data/stationxml_ringlaser/station_BW_ROMY.xml",  # Rotation station inventory
    'path_to_inv_tra': "/home/andbro/kilauea-data/stationxml_ringlaser/station_GR_FUR.xml",  # Translation station inventory
    
    # Path configurations
    'workdir': "./",            # Working directory
    'path_to_data_out': "./output/",  # Output directory

    # Set rotation of components
    'rotate_zne': True,

    # Rotate ROMY components
    'use_romy_zne': False,
    'keep_z': True,

    # Remove responses
    'tra_remove_response': True,
    'rot_remove_response': True,

    # Project information
    'project': "ROMY_Analysis",  # Project name

    # Backazimuth calculation parameters
    'baz_step': 1,               # Step size for backazimuth search in degrees
    'time_win_sec': 30,          # Window length in seconds
    'time_win_overlap': 15,  # Window overlap in seconds

    # Optional parameters
    'verbose': True,            # Print detailed information
}

Setup a sixdegrees object

In [3]:
sd = sixdegrees(conf=config)

Store the configuration as a pickle file

Get key information about the event

In [4]:
ev = sd.get_event_info(
    origin_time=config['origin_time'],
    base_catalog="USGS",
    magnitude=config['magnitude']
)


Found event:
Origin time: 2025-07-29T23:24:52.480000Z
Magnitude: 8.8 mww
Location: 52.498°N, 160.264°E
Depth: 35.0 km
Epicentral Distance: 8469.0 km
Epicentral Distance: 76.2°
Backazimuth: 18.9°


This will load the data from the data source:
- local mseed file
- local sds system
- fdsn (online)

and trim it to the time window

In [5]:
sd.load_data(config['tbeg'], config['tend'], merging=True)

-> fetching GR.FUR..BHZ data from SDS
-> fetching GR.FUR..BHN data from SDS
-> fetching GR.FUR..BHE data from SDS
-> translation inventory provided: /home/andbro/kilauea-data/stationxml_ringlaser/station_GR_FUR.xml
-> removing response: ACC
-> rotating translational data ENZ to ZNE
3 Trace(s) in Stream:
GR.FUR..BHZ | 2025-07-29T23:29:58.989999Z - 2025-08-01T23:30:00.989999Z | 20.0 Hz, 5184041 samples
GR.FUR..BHN | 2025-07-29T23:29:58.989999Z - 2025-08-01T23:30:00.989999Z | 20.0 Hz, 5184041 samples
GR.FUR..BHE | 2025-07-29T23:29:58.989999Z - 2025-08-01T23:30:00.989999Z | 20.0 Hz, 5184041 samples
-> fetching BW.ROMY.10.BJZ data from SDS
-> fetching BW.ROMY..BJU data from SDS


/home/andbro/anaconda3/envs/obs2/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Record with offset=276480 has a fractional second (.0001 seconds) of 10000. This is not strictly valid but will be interpreted as one or more additional seconds.
  warnings.warn(_w, InternalMSEEDWarning)
/home/andbro/anaconda3/envs/obs2/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Record with offset=279040 has a fractional second (.0001 seconds) of 10000. This is not strictly valid but will be interpreted as one or more additional seconds.
  warnings.warn(_w, InternalMSEEDWarning)
/home/andbro/anaconda3/envs/obs2/lib/python3.10/site-packages/obspy/io/mseed/headers.py:823: InternalMSEEDWarning: readMSEEDBuffer(): Record with offset=285696 has a fractional second (.0001 seconds) of 10000. This is not strictly valid but will be interpreted as one or more additional seconds.
  warnings.warn(_w, Internal

-> fetching BW.ROMY..BJV data from SDS
-> rotation inventory provided: /home/andbro/kilauea-data/stationxml_ringlaser/station_BW_ROMY.xml
-> removing sensitivity
3 Trace(s) in Stream:
BW.ROMY..BJU   | 2025-07-29T23:29:59.000001Z - 2025-08-01T23:30:01.000001Z | 20.0 Hz, 5184041 samples
BW.ROMY..BJV   | 2025-07-29T23:29:59.031200Z - 2025-08-01T23:30:00.981200Z | 20.0 Hz, 5184040 samples
BW.ROMY.10.BJZ | 2025-07-29T23:29:58.985200Z - 2025-08-01T23:30:00.985200Z | 20.0 Hz, 5184041 samples


In [6]:
filename = "./config/"+f"config_ROMY_M{str(config['magnitude'])}_{config['tbeg'][:10].replace('-','')}"

sd.store_as_yaml(config, filename)


In [7]:
sd.st.write("./data/romy_eigenmodes.mseed")

/home/andbro/anaconda3/envs/obs2/lib/python3.10/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
